In [33]:
## NOTE: I used many ideas from this article and python notebook:
## https://towardsdatascience.com/machine-learning-with-pyspark-and-amazon-emr-3149dbc847ae
## https://github.com/maleckicoa/Sparkify-Project/blob/master/Sparkify_Big.ipynb

sc.install_pypi_package("pandas==0.25.1") 
sc.install_pypi_package("scikit-learn")

import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
spark = SparkSession \
    .builder \
    .appName("CS643_Project") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
## Load Training Dataset
train_df = spark.read.format('csv').options(header='true', inferSchema='true', sep=';').load('s3a://cs643johndaudelin/TrainingDataset.csv')
validation_df = spark.read.format('csv').options(header='true', inferSchema='true', sep=';').load('s3a://cs643johndaudelin/ValidationDataset.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
train_df.toPandas().head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   """""fixed acidity""""  ...  """"quality"""""
0                     8.9  ...                 6
1                     7.6  ...                 5
2                     7.9  ...                 5
3                     8.5  ...                 5
4                     6.9  ...                 6

[5 rows x 12 columns]

In [46]:
validation_df.toPandas().head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   """fixed acidity""""  ...  """"quality"""""
0                   7.4  ...                 5
1                   7.8  ...                 5
2                   7.8  ...                 5
3                  11.2  ...                 6
4                   7.4  ...                 5

[5 rows x 12 columns]

In [47]:
train_df = train_df.withColumnRenamed('"""""fixed acidity""""', "fixed acidity")\
       .withColumnRenamed('""""volatile acidity""""', "volatile acidity")\
       .withColumnRenamed('""""citric acid""""', "citric acid")\
       .withColumnRenamed('""""residual sugar""""', "residual sugar")\
       .withColumnRenamed('""""chlorides""""', "chlorides")\
       .withColumnRenamed('""""free sulfur dioxide""""', "free sulfur dioxide")\
       .withColumnRenamed('""""total sulfur dioxide""""', "total sulfur dioxide")\
       .withColumnRenamed('""""density""""', "density")\
       .withColumnRenamed('""""pH""""', "pH")\
       .withColumnRenamed('""""sulphates""""', "sulphates")\
       .withColumnRenamed('""""alcohol""""', "alcohol")\
       .withColumnRenamed('""""quality"""""', "label")

validation_df = validation_df.withColumnRenamed('"""fixed acidity""""', "fixed acidity")\
       .withColumnRenamed('""""volatile acidity""""', "volatile acidity")\
       .withColumnRenamed('""""citric acid""""', "citric acid")\
       .withColumnRenamed('""""residual sugar""""', "residual sugar")\
       .withColumnRenamed('""""chlorides""""', "chlorides")\
       .withColumnRenamed('""""free sulfur dioxide""""', "free sulfur dioxide")\
       .withColumnRenamed('""""total sulfur dioxide""""', "total sulfur dioxide")\
       .withColumnRenamed('""""density""""', "density")\
       .withColumnRenamed('""""pH""""', "pH")\
       .withColumnRenamed('""""sulphates""""', "sulphates")\
       .withColumnRenamed('""""alcohol""""', "alcohol")\
       .withColumnRenamed('""""quality"""""', "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sc.install_pypi_package("pyspark2pmml")

In [94]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import RFormula
from pyspark.mllib.evaluation import MulticlassMetrics

formula = RFormula(formula = "label ~ .")
classifier = DecisionTreeClassifier()
pipeline = Pipeline(stages = [formula, classifier])
model = pipeline.fit(train_df)
preds = model.transform(validation_df)
results = preds.select(['prediction', 'label'])

# from pyspark2pmml import PMMLBuilder
# pmmlBuilder = PMMLBuilder(sc, train_df, model).putOption(classifier, "compact", True)
# pmmlBuilder.buildFile("DecisionTreeModel.pmml")

## NEED TO INSTALL https://github.com/jpmml/jpmml-sparkml on EC2's

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'ctx'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/mllib/evaluation.py", line 200, in __init__
    sc = predictionAndLabels.ctx
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1307, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'ctx'



In [78]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

model = LogisticRegressionWithLBFGS.train(train_df, numClasses=10)

# Compute raw scores on the test set
predictionAndLabels = validation_df.map(lambda lp: (float(model.predict(lp.features)), lp.label))

# Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
features
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/mllib/classification.py", line 401, in train
    initialWeights = [0.0] * len(data.first().features) * (numClasses - 1)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 1538, in __getattr__
    raise AttributeError(item)
AttributeError: features



In [56]:
assembler = VectorAssembler(
    inputCols=["fixed acidity",
               "volatile acidity",
               "citric acid",
               "residual sugar",
               "chlorides",
               "free sulfur dioxide",
               "total sulfur dioxide",
               "density",
               "pH",
               "sulphates",
               "alcohol"],
                outputCol="inputFeatures")

scaler = Normalizer(inputCol="inputFeatures", outputCol="features")

lr=LogisticRegression()
rf= RandomForestClassifier()

pipeline1=Pipeline(stages=[assembler, scaler, lr])
pipeline2 = Pipeline(stages=[assembler, scaler, rf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
paramgrid =ParamGridBuilder().build()

evaluator=MulticlassClassificationEvaluator(metricName="f1")

crossval= CrossValidator(estimator=pipeline1,  
                         estimatorParamMaps=paramgrid,
                         evaluator=evaluator, 
                         numFolds=3
                        )

cvModel1=crossval.fit(train_df) 
evaluator.evaluate(cvModel1.transform(validation_df))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.562631807944308

In [58]:
crossval= CrossValidator(estimator=pipeline2,  
                         estimatorParamMaps=paramgrid,
                         evaluator=evaluator, 
                         numFolds=3
                        )

cvModel2=crossval.fit(train_df) 
evaluator.evaluate(cvModel2.transform(validation_df))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.47683839637797576

In [ ]:
from jpmml_sparkml import toPMMLBytes
pmmlBytes = toPMMLBytes(sc, df, pipelineModel)
print(pmmlBytes)